# Exercise2:  Regression analysis using crime dataset
    

In this exercise you need to perform multivariate regression analysis of Communities and Crime dataset for regression
https://archive.ics.uci.edu/ml/datasets/Communities+and+Crime+Unnormalized
We use edited version of the dataset, where only the most relevant features are kept.
Your notebooks must contain not only code cells but also text cells with your analysis and comments.
Try to perform following tasks:

- Load crime dataset (CommViolPredUnnormalizedData_edited.csv)

- Visualize data, comment on what you observe. Does this data need preprocessing?

- Define your predicted values y_crime ('ViolentCrimesPerPop') and feature values X_crime (the rest of the columns)

- Split the data into train and test set

- Decide on type of regression to use

- Perform regression analysis using sklearn without normalization and with normalization. Compare.

- Perform regression without  regularization and with Ridge regularization. Compare.

- Check how model performance depends on alpha parameter of Ridge regularization.